In [ ]:
from datetime import timedelta
from fuzzywuzzy import fuzz

import pandas as pd
import requests

import json
import os

In [13]:
r = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')
bikeshare_stations = json.loads(r.content)['data']['stations']
bikeshare_stations = pd.DataFrame(bikeshare_stations)[['station_id', 'name', 'lat', 'lon']].astype({
    'station_id': 'float64',
})

# bikeshare_stations.to_csv('./data/bikeshare_stations.csv', index=False)

In [14]:
date_formats = {
    'Bikeshare Ridership (2017 Q1).csv': ['%d/%m/%Y %H:%M', -4],
    'Bikeshare Ridership (2017 Q2).csv': ['%d/%m/%Y %H:%M', -4],
    'Bikeshare Ridership (2017 Q3).csv': ['%m/%d/%Y %H:%M', 0],
    'Bikeshare Ridership (2017 Q4).csv': ['%m/%d/%y %H:%M:%S', 0],
}

df = pd.DataFrame()
for fn, fmt in date_formats.items():
    tmp = pd.read_csv(os.path.join('./data/original', fn))
    tmp['trip_start_time'] = pd.to_datetime(tmp['trip_start_time'], format=fmt[0], errors='coerce')
    tmp['trip_start_time'] = tmp['trip_start_time'] + timedelta(hours=fmt[1])

    df = pd.concat([df, tmp], sort=False).reset_index(drop=True)

# df.to_csv('./data/original/ridership_merged_2017.csv', index=False)

In [15]:
stations_start = df[['from_station_id', 'from_station_name']]
stations_end = df[['to_station_id', 'to_station_name']]
stations_start.columns = stations_end.columns = ['station_id', 'name']

stations = pd.concat([stations_start, stations_end]).dropna(how='all').drop_duplicates().reset_index(drop=True)

no_ids = stations[stations['station_id'].isnull()]

In [16]:
for idx, miss in no_ids.iterrows():
    max_score = 0
    
    for i, exist in bikeshare_stations[['station_id', 'name']].iterrows():
        score = fuzz.ratio(miss['name'], exist['name'])
        
        if score > 80 and score > max_score:
            max_score = score
            no_ids.at[idx, 'station_id'] = exist['station_id']
    
    if max_score <= 80:
        print('WARN: {0} station could not be matched to an existing station'.format(miss['name']))

no_ids = no_ids.dropna()

WARN: Michael Sweet Ave / St. Patrick St station could not be matched to an existing station
WARN: Bremner Blvd / Spadina Ave station could not be matched to an existing station
WARN: Bay St / Bloor St W station could not be matched to an existing station
WARN: Bathurst St / Queens Quay W station could not be matched to an existing station
WARN: Davenport Rd / Bedford Rd station could not be matched to an existing station
WARN: Roxton Rd / College St station could not be matched to an existing station
WARN: Base Station station could not be matched to an existing station
WARN: Fringe Next Stage - 7219 station could not be matched to an existing station


In [17]:
stations = pd.concat([stations[~stations['station_id'].isnull()], no_ids])\
             .merge(bikeshare_stations[['station_id', 'lat', 'lon']], how='left', on='station_id')\
             .drop_duplicates()

In [18]:
df = df.merge(stations, how='inner', left_on='from_station_name', right_on='name') \
       .merge(stations, how='inner', left_on='to_station_name', right_on='name', suffixes=['_from', '_to']) \
       .drop_duplicates()

df = df[[x for x in df.columns if not x.endswith('_station_id') and not x.endswith('_station_name') and x != 'trip_stop_time']]

# df.to_csv('./data/bikeshare_ridership_2017.csv', index=False)

In [19]:
df[[x for x in df.columns if 'id' not in x]].describe()

,trip_duration_seconds,lat_from,lon_from,lat_to,lon_to
count,1.463724e+06,1.463724e+06,1.463724e+06,1.463724e+06,1.463724e+06
mean,1.020607e+03,4.365378e+01,-7.939055e+01,4.365298e+01,-7.938999e+01
std,9.861973e+03,1.069626e-02,2.016864e-02,1.034679e-02,2.031100e-02
min,1.000000e+00,4.362671e+01,-7.947657e+01,4.362671e+01,-7.947657e+01
25%,4.140000e+02,4.364561e+01,-7.940146e+01,4.364561e+01,-7.939960e+01
50%,6.550000e+02,4.365282e+01,-7.938758e+01,4.365212e+01,-7.938750e+01
75%,1.022000e+03,4.366207e+01,-7.937990e+01,4.366041e+01,-7.937988e+01
max,6.382030e+06,4.369402e+01,-7.928898e+01,4.369402e+01,-7.928898e+01


In [20]:
df = df[~((df['trip_duration_seconds'] < 30)&(df['station_id_from']==df['station_id_to']))]

In [21]:
df = df[(df['trip_duration_seconds']>2)]

In [22]:
df.describe()

,trip_id,trip_duration_seconds,station_id_from,lat_from,lon_from,station_id_to,lat_to,lon_to
count,1.437122e+06,1.437122e+06,1.437122e+06,1.437122e+06,1.437122e+06,1.437122e+06,1.437122e+06,1.437122e+06
mean,1.545465e+06,1.039058e+03,7.094210e+03,4.365382e+01,-7.939054e+01,7.093333e+03,4.365300e+01,-7.938997e+01
std,4.790821e+05,9.951889e+03,7.633668e+01,1.067885e-02,2.011516e-02,7.608693e+01,1.032546e-02,2.025898e-02
min,7.123820e+05,2.700000e+01,7.000000e+03,4.362671e+01,-7.947657e+01,7.000000e+03,4.362671e+01,-7.947657e+01
25%,1.131567e+06,4.270000e+02,7.032000e+03,4.364561e+01,-7.940146e+01,7.033000e+03,4.364561e+01,-7.939960e+01
50%,1.546666e+06,6.650000e+02,7.068000e+03,4.365282e+01,-7.938758e+01,7.067000e+03,4.365212e+01,-7.938750e+01
75%,1.958050e+06,1.031000e+03,7.158000e+03,4.366207e+01,-7.937990e+01,7.158000e+03,4.366041e+01,-7.937988e+01
max,2.383646e+06,6.382030e+06,7.289000e+03,4.369402e+01,-7.928898e+01,7.289000e+03,4.369402e+01,-7.928898e+01


In [23]:
df.shape

(1437122, 12)

In [24]:
# Calculate the outlier in terms of trip durations (see https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba)

q1 = df['trip_duration_seconds'].quantile(0.25)
q3 = df['trip_duration_seconds'].quantile(0.75)
interquartile_range = q3 - q1

df = df[~((df['trip_duration_seconds'] < (q1 - 1.5 * interquartile_range))|(df['trip_duration_seconds'] > (q3 + 1.5 * interquartile_range)))].reset_index(drop=True)

In [25]:
df.shape

(1354224, 12)

In [26]:
df.to_csv('./data/bikeshare_ridership_2017_v2.csv', index=False)